# Single Source Shortest Path (SSSP)

In this notebook, we will use cuGraph to compute the shortest path from a starting vertex to everyother vertex in our training dataset.

Notebook Credits
* Original Authors: Bradley Rees and James Wyles
* Last Edit: 08/09/2019

RAPIDS Versions: 0.9.0   

Test Hardware

* GV100 32G, CUDA 10.0



## Introduction

Single source shortest path computes the shortest paths from the given starting vertex to all other reachable vertices. 

To compute SSSP for a graph in cuGraph we use:
**cugraph.sssp(G, source)**

Input
* __G__: cugraph.Graph object
* __source__: int, Index of the source vertex

Returns 
* __df__:  a cudf.DataFrame object with two columns:
    * df['vertex']: The vertex identifier for the vertex
    * df['distance']: The computed distance from the source vertex to this vertex


## cuGraph 0.7 Notice 
cuGraph version 0.7 has some limitations:
* Only Int32 Vertex ID are supported
* Only float (FP32) edge data is supported
* Vertex numbering is assumed to start at zero

These limitations are being addressed and will be fixed future versions.  
These example notebooks will illustrate how to manipulate the data so that it comforms to the current limitations    
    



### Test Data
We will be using the Zachary Karate club dataset 
*W. W. Zachary, An information flow model for conflict and fission in small groups, Journal of
Anthropological Research 33, 452-473 (1977).*


![Karate Club](./img/zachary_black_lines.png)

This is a small graph which allows for easy visual inspection to validate results.  
__Note__: The Karate dataset starts with vertex ID 1 which the cuGraph analytics assume a zero-based starting ID.  

In [1]:
# Import needed libraries
import cugraph
import cudf
from collections import OrderedDict

### Read the data and adjust the vertex IDs

In [2]:
# Test file  - using the clasic Karate club dataset.  
datafile='./data/karate-data.csv'

In [3]:
# Read the data file
cols = ["src", "dst"]

dtypes = OrderedDict([
        ("src", "int32"), 
        ("dst", "int32")
        ])

gdf = cudf.read_csv(datafile, names=cols, delimiter='\t', dtype=list(dtypes.values()) )

In [4]:
# Need to shift the vertex IDs to start with zero rather than one (next version of cuGraph will fix this issue)
gdf["src_0"] = gdf["src"] - 1
gdf["dst_0"] = gdf["dst"] - 1

In [5]:
gdf.head().to_pandas()

,src,dst,src_0,dst_0
0,1,2,0,1
1,1,3,0,2
2,1,4,0,3
3,1,5,0,4
4,1,6,0,5


### Create a Graph and call SSSP

In [6]:
# create a Graph 
G = cugraph.Graph()
G.add_edge_list(gdf["src_0"], gdf["dst_0"])

In [7]:
# Call cugraph.sssp to get the distances from vertex 0:
df = cugraph.sssp(G, 0)

In [8]:
# Find the farthest vertex from the source using the distances:
# __note__ the vertex ID is shifted back to 1-based so that it can be seen on picture above
bestDist = df['distance'][0]
bestVert = df['vertex'][0]
for i in range(len(df)):
    if df['distance'][i] > bestDist:
        bestDist = df['distance'][i]
        bestVert = df['vertex'][i]
print("Farthest vertex is " + str(bestVert + 1) + " with distance of " + str(bestDist))

Farthest vertex is 15 with distance of 3.0


There are a number of vertices with the same distance of 3

___
Copyright (c) 2019, NVIDIA CORPORATION.

Licensed under the Apache License, Version 2.0 (the "License");  you may not use this file except in compliance with the License. You may obtain a copy of the License at http://www.apache.org/licenses/LICENSE-2.0

Unless required by applicable law or agreed to in writing, software distributed under the License is distributed on an "AS IS" BASIS, WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied. See the License for the specific language governing permissions and limitations under the License.
___